In [ ]:
# default_exp utils

In [ ]:
%load_ext autoreload
%autoreload 2

# create references

> API details.

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
# export
import os
import sys
import re
import argparse
import ftplib
from create_reference import defaults

In [ ]:
# export
def get_args(*args):
    parser = argparse.ArgumentParser(prog='fetchr',
                                     description='Fetch and Generate references for bioinformatics analysis',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter
                                    )
    
    parser.add_argument('--species',nargs='+',choices=defaults.species,default='homo_spaiens',
                       help='Reference or references from which species')
    parser.add_argument('--indexs',nargs='+',choices=defaults.softwares,default='samtools',
                       help='Using which software(s) to create indexes')
    parser.add_argument('--reference-version','-rv',default=99,type=int,
                       help='For homo_spaiens, version=75 is the last version of Grch37 reference, \
                       you can check the version in ftp://ftp.ensembl.org/pub/')
    parser.add_argument('--outdir','-o',default='./',
                       help='Reference and indexes generated direction')
    parser.add_argument('--workflow','-w',default='sqlite:////sqlite.db',
                       help='Workflow path')
    parser.add_argument('--thread','-t',default=4,
                       help='Thread number')
    return parser.parse_args(args)


In [ ]:
args=get_args('-h')

usage: fetchr [-h]
              [--species {homo_sapiens,mus_musculus,rattus_norvegicus,caenorhabditis_elegans,equus_caballus,capra_hircus,drosophila_melanogaster,macaca_mulatta,ovis_aries,sus_scrofa,oryctolagus_cuniculus,bos_taurus,saccharomyces_cerevisiae,gallus_gallus,gasterosteus_aculeatus,danio_rerio,oryzias_latipes,anser_cygnoides,callithrix_jacchus,pan_troglodytes,pongo_abelii,meleagris_gallopavo,oreochromis_niloticus,bos_grunniens,phascolarctos_cinereus,salmo_salar,amphilophus_citrinellus,ciona_intestinalis,monodelphis_domestica,mesocricetus_auratus,scophthalmus_maximus,felis_catus,microcebus_murinus,cynoglossus_semilaevis,anolis_carolinensis,ailuropoda_melanoleuca,melopsittacus_undulatus,clupea_harengus,neolamprologus_brichardi,maylandia_zebra,haplochromis_burtoni,pundamilia_nyererei,larimichthys_crocea,chlorocebus_sabaeus} [{homo_sapiens,mus_musculus,rattus_norvegicus,caenorhabditis_elegans,equus_caballus,capra_hircus,drosophila_melanogaster,macaca_mulatta,ovis_aries,sus_scr

SystemExit: 0

/Users/logan/.conda/envs/nbdev-cd/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# export 

def file_exists(f):
    return os.path.exists(f)

In [ ]:
file_exists('./00_utils.ipynb')


True

In [ ]:
# export

def get_ffp(species):
    return re.compile(defaults.fasta_file_pattern.format(species=species),re.IGNORECASE)

def get_gfp(species):
    return re.compile(defaults.gtf_file_pattern.format(species=species),re.IGNORECASE)

In [ ]:
# export

def get_likely_file_from_ftp(ftp,version,species,ftype,dtype,pattern):
    if ftype == 'fasta':
        direction='/pub/release-{version}/{ftype}/{species}/{dtype}/'.format(
            version=version,
            species=species,
            ftype=ftype,
            dtype=dtype)
    elif ftype == 'gtf':
        direction='/pub/release-{version}/{ftype}/{species}/'.format(
            version=version,
            species=species,
            ftype=ftype)
    try:
       
        ftp.cwd(direction)
        files = ftp.nlst()
        for f in files:
#                 print(f)
            if len(pattern.findall(f))>0:
                return 'ftp://'+ftp_server+direction+f
        raise ValueError('No fit ' + ftype +' file in ftp://'+ftp_server+direction)
    except ftplib.all_errors as e:
        print(e)
        sys.exit(1)
        

In [ ]:
with ftplib.FTP(defaults.ensembl_ftp) as ftp:
    ftp.login()
    for s in defaults.species:
        print(s)
        print(get_likely_file_from_ftp(ftp,'99',s,'fasta','dna',get_ffp(s)))
        print(get_likely_file_from_ftp(ftp,'99',s,'gtf',None,get_gfp(s)))
    

homo_sapiens


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()